In [17]:
import shutil
import os
import pandas as pd
import datetime
import pickle
from tqdm import tqdm

In [18]:
class pandasmaker:
    
    def __init__(self, dir_):
        self.dir_ = dir_
        self.num_list = [name for name in os.listdir(self.dir_) if os.path.isfile(os.path.join(self.dir_, name))]
        
    def make_pandas(self):
        pieces = []
        columns = ['URL', 'Title', 'Visitors', 'Delta', 'Shift', 'ID']
        for date in tqdm(self.num_list):
            path = self.dir_ + '%s' % date
            frame = pd.read_table(path, header=None, usecols=[0,1,2,3,4,5], skiprows=1, names=columns)
            frame ['Date'] = date
            pieces.append(frame) # Собрать все данные в один объект DataFrame
            names = pd.concat(pieces, ignore_index=True, sort=False)
        
        names['Locate'] = names['Date'].apply(lambda x : x[:2])
        names['Date'] = pd.to_datetime(names['Date'].apply(lambda x : x[3:13] + ' ' + x[14:-4].replace('_', ':')))
        names['Source_of_info'] = names.apply(
            lambda row: '.'.join([i for i in row['URL'].split('/')[0].split('.') if i != 'www']), axis=1)    
        
        name_str = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S') + '.pic'
        names.to_pickle(name_str)
        shutil.rmtree(self.dir_, ignore_errors=True)